### Import thư viện

In [ ]:
import numpy as np
import pandas as pd
import time
import math
from numba import cuda
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')

### Cài đặt phiên bản XGBoost song song

Thực hiện song song hoá một số hàm có khả năng song song nhằm triển khai XGBoost với thời gian thực thi ngắn hơn so với phiên bản tuần tự ban đầu trong khi mô hình vẫn duy trì độ chính xác chấp nhận được

Mục tiêu chính là song song hoá thành công hàm `find_best_split()` vì đây là hàm chiếm hơn 90% tổng thời gian chạy XGBoost, nếu song song hoá tốt hàm này, thời gian thực thi tổng thể sẽ giảm đi rất nhiều. Ngoài ra, nhóm cũng sẽ tiến hành song song hoá một số hàm tính toán khác với mong muốn giảm thời gian huấn luyện đến mức thấp nhất.

### Xét class **`Tree`**

**Song song hóa hàm `find_best_split()`**

**Mô tả hàm**: `find_best_split()` là hàm tìm giá trị split sao cho gain của chúng khi rẽ nhánh đạt giá trị lớn nhất => đây là giá trị split tốt nhất dùng để rẽ nhánh.

**Ý tưởng tuần tự**: Vét cạn tất cả trường hợp: Xét feature thứ i, tìm từng giá trị split theo feature i, tính gain và so sánh với `best_gain`, lặp lại đến hết và sau cùng lưu vị trí feature cùng với giá trị split và best_gain.

**Ý tưởng song song**: Chia làm 2 hàm con:
- `compute_split_value()`: tính tất cả giá trị split value, mỗi thread sẽ đảm nhiệm tính một giá trị split
- `compute_gain()`: sau khi xác định được mảng giá trị split, với mỗi giá trị, ta tiến hành rẽ nhánh và tính gain.
  
Kết hợp với `np.argmax()` của thư viện numpy để tìm được giá trị split tốt nhất. Tương tự như ý tưởng tuần tự, ta lưu vị trí feature cùng với giá trị split và best_gain.

In [ ]:
''' Hàm tính tất cả giá trị split value. Mỗi thread sẽ tìm split value của 2 giá trị liên tiếp
    Nếu giá trị tại feature thứ i của mẫu A và B bằng nhau -> trả giá trị nan
    Nếu giá trị tại feature thứ i của mẫu A và B khác nhau -> trả giá trị mean của 2 feature thuộc 2 mẫu
    Tham số
        input: ma trận dữ liệu X
        output: ma trận split value

    Trước khi đưa vào input sẽ được sort axis = 0 -> sort lại theo từng feature để dễ dàng tìm split value
'''

@cuda.jit
def compute_split_value(input, output):
    c, r = cuda.grid(2)
    if r < output.shape[0] and c < output.shape[1]:
        if input[r, c] != input[r + 1, c]:
            split = (input[r, c] + input[r + 1, c]) / 2
            output[r, c] = split
        else:
            output[r, c] = np.nan

In [ ]:
''' Tính gain ứng với giá trị split. Một thread sẽ thực hiện việc chia data thành 2 bên left right & tính gain
    Với mỗi giá trị split:
    Nếu split value là nan thì bỏ qua, trả kết quả gain = nan
    Nếu split value khác nan: tính residual và prob của 2 phần left right, xét tính hợp lệ của cover, tính gain
    Trả kết quả về
'''

@cuda.jit
def compute_gain(split, output, X, residuals, p, lambda_, min_child_weight, min_samples, root_gain):
    c, r = cuda.grid(2)
    if r < split.shape[0] and c < split.shape[1]:
        if split[r, c] != np.nan:
            left_nu, right_nu, left_de, right_de, left_samples, right_samples = 0, 0, 0, 0, 0, 0
            for i in range(X.shape[0]):
                if X[i, c] <= split[r, c]:
                    left_nu += residuals[i]
                    left_de += p[i] * (1 - p[i])
                    left_samples += 1
                else:
                    right_nu += residuals[i]
                    right_de += p[i] * (1 - p[i])
                    right_samples += 1
            if (left_de < min_child_weight or right_de < min_child_weight
                or left_samples < min_samples or right_samples < min_samples):
                output[r,c] = np.nan
            else:
                left_sim = (left_nu ** 2) / (left_de + lambda_)
                right_sim = (right_nu ** 2) / (right_de + lambda_)
                gain = left_sim + right_sim - root_gain
                output[r,c] = gain
        else:
            output[r,c] = np.nan

In [ ]:
class Tree:
    def __init__(self, max_depth = 3, min_samples = 1, min_child_weight = 1, lambda_ = 0, gamma = 0):
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.min_child_weight = min_child_weight
        self.lambda_ = lambda_
        self.gamma = gamma
        self.tree = {}
        self.fbs_time = 0

    def similarity(self, residual, probs):
        nu = np.sum(residual) ** 2
        de = np.sum(probs * (1 - probs)) + self.lambda_
        return nu / de

    def compute_output(self, residual, probs):
        nu = np.sum(residual)
        de = np.sum(probs * (1 - probs)) + self.lambda_
        return nu / de

    def split_data(self, X, feature_idx, split_value):
        left_idx = X[:, feature_idx] <= split_value
        right_idx = X[:, feature_idx] > split_value
        return left_idx, right_idx

    ''' Find_best_split được song song hóa sử dụng 2 hàm con. Các bước:
        1. Tạo mảng split array - các giá trị để chia sample thành 2 bên trái phải
            Các giá trị nan, tượng trưng cho việc đã có trường hợp split này
        2. Tính gain cho mỗi trường hợp split
        3. Gán các giá trị nan thành -inf để loại trường hợp nan
        4. Transpose mảng gain:
            + Do phiên bản tuần tự dò tìm max đi theo từng cột
            + Phiên bản song song dùng np.argmax() do tìm max theo từng hàng
            + Vì thế cần transpose mảng gain để cả 2 phiên bản đều tìm cùng 1 kết quả
        5. Dùng np.argmax() để tìm được vị trí có gain lớn nhất
            Dùng kết quả để xác định lại vị trí thì phải đảo ngược result = (max_idx[1], max_idx[0])
        6. Xem xét tỉa cây dựa trên gamma
    '''
    def find_best_split(self, X, residuals, probs):
        best_gain = -np.inf
        best_split_feature_idx = None
        best_split_value = None

        split_array = np.empty((X.shape[0] - 1, X.shape[1]))
        block_size = (32, 32)
        grid_split = (math.ceil(split_array.shape[1] / block_size[0]),
                      math.ceil(split_array.shape[0] / block_size[1]))
        compute_split_value[grid_split, block_size](np.sort(X, axis=0), split_array)

        root_gain = self.similarity(residuals, probs)
        gain_array = np.empty((split_array.shape[0], split_array.shape[1]))
        grid_gain = (math.ceil(gain_array.shape[1] / block_size[0]),
                     math.ceil(gain_array.shape[0] / block_size[1]))
        compute_gain[grid_gain, block_size](split_array, gain_array, X, residuals,
                                            probs, self.lambda_, self.min_child_weight, self.min_samples, root_gain)

        gain_array[np.isnan(gain_array)] = -np.inf

        if np.sum(gain_array == -np.inf) != (gain_array.shape[0] * gain_array.shape[1]):
            tmp_gain = np.transpose(gain_array)
            max_idx = np.unravel_index(tmp_gain.argmax(), tmp_gain.shape)
            final_idx = (max_idx[1], max_idx[0])
            best_split_feature_idx = final_idx[1]
            best_gain = gain_array[final_idx]
            best_split_value = split_array[final_idx]

        if(best_gain - self.gamma < 0):
            best_split_feature_idx = None
            best_split_value = None

        return best_split_feature_idx, best_split_value

    def build_tree(self, X, residual, probs, depth):
        if depth >= self.max_depth or len(X) <= self.min_samples:
            return self.compute_output(residual, probs)

        start = time.time()
        split_feature_idx, split_value = self.find_best_split(X, residual, probs)
        end = time.time()
        self.fbs_time += (end - start)

        if split_feature_idx is None:
            return self.compute_output(residual, probs)

        left_idx, right_idx = self.split_data(X, split_feature_idx, split_value)
        left = self.build_tree(X[left_idx], residual[left_idx], probs[left_idx], depth + 1)
        right = self.build_tree(X[right_idx], residual[right_idx], probs[right_idx], depth + 1)

        self.tree = {
            'split_feature_idx': split_feature_idx,
            'split_value': split_value,
            'left_child': left,
            'right_child': right
        }
        return self.tree

    def get_output(self, x, tree):
        if isinstance(tree, dict):
            split_feature_idx = tree['split_feature_idx']
            split_value = tree['split_value']
            if x[split_feature_idx] <= split_value:
                return self.get_output(x, tree['left_child'])
            else:
                return self.get_output(x, tree['right_child'])
        else:
            return tree

    def fit(self, X, residual, probs):
        depth = 0
        self.tree = self.build_tree(X, residual, probs, depth)

    def predict(self, X):
        return np.array([self.get_output(x, self.tree) for x in X])

### Xét class **`XGBoost`**

**Song song hóa hàm `residual()`**

**Mô tả hàm**: `residual()` là hàm tính toán sự chênh lệch giữa y thực tế và kết quả dự đoán

**Ý tưởng tuần tự**: Sử dụng thư viện numpy để tính giá trị residual  

**Ý tưởng song song**: Mỗi một thread sẽ được dùng để tính giá trị residual của một mẫu (sample)

In [ ]:
''' Hàm tính toán sự chênh lệch giữa y thực tế và kết quả dự đoán
    y_true, y_pred lần lượt là giá trị y thực tế và kết quả dự đoán
    residual là kết quả chênh lệch giữa y_true và y_pred
'''

@cuda.jit
def residual_kernel(y_true, y_pred, residual):
    i = cuda.grid(1)
    if i < residual.shape[0]:
        residual[i] = y_true[i] - y_pred[i]

**Song song hóa hàm `compute_logodds()`**

**Mô tả hàm**: compute_logodds() là hàm chuyển giá trị xác suất p thành giá trị log(odds)

**Ý tưởng tuần tự**: Sử dụng thư viện numpy để tính giá trị log(odds)

**Ý tưởng song song**: Mỗi một thread sẽ được dùng để tính giá trị log(odds) của một mẫu (sample)

In [ ]:
''' Hàm chuyển giá trị xác suất p thành giá trị log(odds)
    p: giá trị xác suất ban đầu
    log_odds: giá trị sau khi chuyển
'''

@cuda.jit
def compute_logodds_kernel(p, log_odds):
    i = cuda.grid(1)
    if i < log_odds.shape[0]:
        log_odds[i] = math.log(p[i] / (1 - p[i]))

**Song song hóa hàm `compute_prob()`**

**Mô tả hàm**: compute_prob() là hàm chuyển giá trị log(odds) thành giá trị xác suất p   

**Ý tưởng tuần tự**: Sử dụng thư viện numpy để tính giá trị xác suất p

**Ý tưởng song song**: Mỗi một thread sẽ được dùng để tính giá trị xác suất p của một mẫu (sample)

In [ ]:
''' Hàm chuyển giá trị log(odds) thành giá trị xác suất p
    logodds_p: giá trị log(odds) ban đầu
    p: giá trị xác suất p sau khi chuyển
'''

@cuda.jit
def compute_prob_kernel(logodds_p, p):
    i = cuda.grid(1)
    if i < p.shape[0]:
        p[i] = math.exp(logodds_p[i]) / (1 + math.exp(logodds_p[i]))

In [ ]:
class XGBoost:
    def __init__(self, n_estimators, lr, lambda_ = 1e-7, gamma = 0, min_child_weight = 1, max_depth = 3):
        self.n_estimators = n_estimators
        self.lr = lr
        self.initial_pred = 0.5
        self.lambda_ = lambda_
        self.min_child_weight = min_child_weight
        self.max_depth = max_depth
        self.gamma = gamma
        self.models = []
        self.fbs_time = 0
        self.logodds_time = 0
        self.residual_time = 0
        self.predict_time = 0

    def fit(self, X, y):
        p = np.full(len(y), self.initial_pred)
        block_size = 32
        grid_size = math.ceil(len(y)/block_size)

        for _ in range(self.n_estimators):
            probs = np.copy(p)

            residual = np.empty(len(y))
            start = time.time()
            residual_kernel[grid_size, block_size](y, p, residual)
            end = time.time()
            self.residual_time += (end - start)

            model = Tree(lambda_ = self.lambda_, gamma = self.gamma, max_depth = self.max_depth, min_child_weight = self.min_child_weight)
            model.fit(X, residual, probs)
            self.fbs_time += model.fbs_time

            log_odds = np.empty(len(y))
            start = time.time()
            compute_logodds_kernel[grid_size, block_size](p, log_odds)
            end = time.time()
            self.logodds_time += (end - start)

            p = np.empty(len(y))
            start = time.time()
            logodds_p = log_odds + self.lr * model.predict(X)
            compute_prob_kernel[grid_size, block_size](logodds_p, p)
            end = time.time()
            self.predict_time += (end - start)

            self.models.append(model)

    def predict_proba(self, X):
        pred = np.full(len(X), self.initial_pred)
        block_size = 32
        grid_size = math.ceil(len(X)/block_size)
        for model in self.models:
            log_odds = np.empty(len(X))
            compute_logodds_kernel[grid_size, block_size](pred, log_odds)
            logodds_p = log_odds + self.lr * model.predict(X)
            pred = np.empty(len(X))
            compute_prob_kernel[grid_size, block_size](logodds_p, pred)
        return pred

### Xây dựng mô hình đa lớp

In [ ]:
class MultiClassifier:
    def __init__(self, n_estimators = 3, lr = 0.3):
        self.models = []
        self.n_estimators = n_estimators
        self.lr = lr
        self.training_time = 0

    def fit(self, X, y):
        start_time = time.time()
        for label in np.unique(y):
            binary_labels = (y == label).astype(int)
            model = XGBoost(self.n_estimators, self.lr)
            model.fit(X, binary_labels)
            self.models.append(model)
        end_time = time.time()
        self.training_time += (end_time - start_time)

    def predict(self, X):
        preds = []
        for model in self.models:
            preds.append(model.predict_proba(X))
        return np.argmax(preds, axis = 0)

### Chạy mô hình

In [ ]:
train = np.load('train_data_3labels.npz', allow_pickle = True)
X_train = train['data']
y_train = train['label']

test = np.load('test_data_3labels.npz', allow_pickle = True)
X_test = test['data']
y_test = test['label']

#### Binary classification

In [ ]:
binary_classifier = XGBoost(n_estimators = 3, lr = 0.3)
start_time = time.time()
binary_classifier.fit(X_train, (y_train == 0).astype(int))
end_time = time.time()

y_prob_pred = binary_classifier.predict_proba(X_test)
binary_labels_pred = (y_prob_pred > 0.5).astype(int)
binary_labels_test = (y_test == 0).astype(int)

accuracy_binary = accuracy_score(binary_labels_test, binary_labels_pred)
time_binary = end_time - start_time

print('Accuracy:', accuracy_binary)
print(f'Total time: {time_binary} seconds')

Accuracy: 0.9466666666666667
Total time: 4.270538568496704 seconds


#### Multi classification

In [ ]:
multi_classifier = MultiClassifier()
multi_classifier.fit(X_train, y_train)

y_pred = multi_classifier.predict(X_test)
accuracy_3labels = accuracy_score(y_test, y_pred)
time_3labels = multi_classifier.training_time

print('Accuracy:', accuracy_3labels)
print(f'Total time: {time_3labels} seconds')

Accuracy: 0.9566666666666667
Total time: 7.5918519496917725 seconds


#### Multi classification trên tập dữ liệu hoàn chỉnh (10 lớp)

In [ ]:
train = np.load('train_data.npz', allow_pickle = True)
X_train = train['data']
y_train = train['label']

test = np.load('test_data.npz', allow_pickle = True)
X_test = test['data']
y_test = test['label']

In [ ]:
multi_classifier = MultiClassifier(n_estimators = 35, lr = 0.3)
multi_classifier.fit(X_train, y_train)

y_pred = multi_classifier.predict(X_test)
accuracy_10labels = accuracy_score(y_test, y_pred)
time_10label = multi_classifier.training_time

print('Accuracy:', accuracy_10labels)
print(f'Total time: {time_10label} seconds')

Accuracy: 0.891
Total time: 2727.113727092743 seconds


### Lưu kết quả

In [ ]:
result_df = pd.read_csv('result.csv')

In [ ]:
result_df['Parallel_training_time'] = [time_binary, time_3labels, time_10label]
result_df['Parallel_accuracy'] = [accuracy_binary, accuracy_3labels, accuracy_10labels]

In [ ]:
result_df.to_csv('result.csv', index = False)